In [12]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import math

from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from scipy import signal
from scipy.fft import fftshift

In [16]:
LOAD_MODEL_PATH = f'../../data/models/enter_v1'
# LOAD_MODEL_PATH = f'../../data/models/leave_v1'

model = keras.models.load_model(f'{LOAD_MODEL_PATH}/model')
params = pd.read_csv(f'{LOAD_MODEL_PATH}/params.csv')
history = pd.read_csv(f'{LOAD_MODEL_PATH}/history.csv')
HISTORY_SIZE = params['history_size'][0]
HORIZON_SIZE = params['horizon_size'][0]
# leave_std = params['y_std'][0]
enter_std = params['y_std'][0]
search_std = params['search_std'][0]

data_norm = pd.read_csv(f'{LOAD_MODEL_PATH}/dataset.csv', parse_dates=['dt']).set_index('dt')

In [21]:
enter_test = pd.read_csv('../../data/marking/enter-test-aggregate.csv', parse_dates=['dt'], index_col='dt')
enter_test_smooth = enter_test.rolling(7, center=True).mean()
enter_test_smooth['enter_cnt_test'] = enter_test_smooth['enter_cnt']
del enter_test_smooth['enter_cnt']

In [19]:
px.line(history, y=['loss', 'val_loss'], log_y=True, log_x=True).show()
px.line(history, y=['accuracy', 'val_accuracy'], log_y=True, log_x=True).show()

In [25]:
x_shape = (len(data_norm.columns), HISTORY_SIZE)
x_elm = x_shape[0] * x_shape[1]
y_shape = (1, HORIZON_SIZE)
y_elm = y_shape[0] * y_shape[1]

def test_model(offset):
    out = data_norm.copy()
    tx = out.copy().iloc[len(data_norm) - offset - HISTORY_SIZE:len(data_norm) - offset]
    # tx['enter_cnt'] /= enter_std
    # tx['leave_cnt'] /= leave_std
    # tx['search_cnt'] /= search_std
    mx = tx.values.reshape((1, x_elm))
    my = model.predict(mx)
    my = my.reshape(y_shape).transpose()
    ty = pd.DataFrame(my)
    ty_index = pd.date_range(tx.index[-1] + pd.DateOffset(1), tx.index[-1] + pd.DateOffset(HORIZON_SIZE), freq='D')
    ty = pd.DataFrame({ 'dt': ty_index, 'enter_cnt': ty[0] }).set_index('dt')
    # ty = pd.DataFrame({ 'dt': ty_index, 'leave_cnt': ty[0] }).set_index('dt')
    ty['enter_cnt'] *= enter_std
    out['enter_cnt'] *= enter_std
    out = pd.concat([out, enter_test_smooth])
    out = out.merge(ty, on='dt', how='outer', suffixes=['', '_model'])

    res = out
    px.line(res, y=['enter_cnt', 'enter_cnt_model', 'enter_cnt_test'], title=f"offset = {offset}").show()
    # px.line(res, y=['leave_cnt', 'leave_cnt_model'], title=f"offset = {offset}").show()

test_model(0*7)
test_model(5*7)
test_model(10*7)
test_model(15*7)
test_model(20*7)
test_model(25*7)
test_model(30*7)
test_model(35*7)

1/1 [==============================] - 0s 14ms/step


1/1 [==============================] - 0s 21ms/step


1/1 [==============================] - 0s 21ms/step


1/1 [==============================] - 0s 22ms/step


1/1 [==============================] - 0s 21ms/step


1/1 [==============================] - 0s 21ms/step


1/1 [==============================] - 0s 20ms/step


1/1 [==============================] - 0s 19ms/step
